In [ ]:
import os
import urllib
from bs4 import BeautifulSoup
import pickle

def fetch_page(url, cache_dir="cache"):
    """Fetches the content of a web page, using caching for efficiency.

    This function retrieves the HTML content of a given URL. It first checks
    if the content is already cached in a local file. If it is, the cached
    content is loaded and returned. Otherwise, the function fetches the content
    from the web using a user-agent header and stores it in the cache directory
    before returning it.

    Args:
        url (str): The URL of the web page to fetch.
        cache_dir (str, optional): The directory to store cached web pages.
            Defaults to "cache".

    Returns:
        BeautifulSoup: The parsed HTML content of the web page.
    """

    cache_file = os.path.join(cache_dir, f"{url.replace('/', '_')}.pkl")
    if os.path.exists(cache_file):
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    else:
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req, timeout=300) as response:
            soup = BeautifulSoup(response, 'lxml')  # Using lxml parser
            os.makedirs(cache_dir, exist_ok=True)
            with open(cache_file, 'wb') as f:
                pickle.dump(soup, f)
            return soup

In [ ]:
import os
import calendar
import concurrent.futures

def get_movies_per_date(year, month, cache_dir="tmp/cache"):
    """
    Get the number of movies released for each date in the specified month, and save the links in the specified format.

    Args:
    - year (int): The year for which to get the movies.
    - month (int): The month for which to get the movies.

    Returns:
    - movies_per_date (dict): A dictionary containing the number of movies released for each date in the month.
    """
    movies_per_date = {}
    num_days = calendar.monthrange(year, month)[1]
    month_dir = os.path.join("movie_links", str(year), str(month).zfill(2))
    os.makedirs(month_dir, exist_ok=True)

    def write_url_to_file(file_name, url):
        with open(file_name, 'a') as f:
            f.write(url + '\n')

    for day in range(1, num_days + 1):
        date = f"{year}-{month:02d}-{day:02d}"
        base_url = f"https://www.imdb.com/search/title/?title_type=feature&release_date={date},{date}&user_rating=1,10&genres=!documentary,!short&primary_language=en&adult=include"
        file_name = os.path.join(month_dir, f"{date}.txt")
        try:
            soup = fetch_page(base_url, cache_dir)
            a_tags = soup.find_all('a', class_='ipc-title-link-wrapper')
            total_links = len(a_tags)
            movies_per_date[date] = total_links
            if total_links != 0:
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    for a_tag in a_tags:
                        href_value = a_tag.get('href')
                        movie_id = href_value.split('/')[2]
                        movie_url = f'https://www.imdb.com/title/{movie_id}/'
                        executor.submit(write_url_to_file, file_name, movie_url)
                        
        except Exception as e:
            print(f"An error occurred: {e}")
            movies_per_date[date] = 0

    return movies_per_date


In [ ]:
import time
month_dict = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

start_year = 1990
end_year = 2000
DIR = f'tmp'
os.makedirs(DIR, exist_ok=True)

start_time = time.time()

for year in range(start_year, end_year + 1):
    for month in month_dict:
        month_str = f"{month:02d}"
        movies_released = get_movies_per_date(year, month)
        with open(f'{DIR}/movies_released_{year}.txt', 'a') as f:
            f.write(f'{month_dict[month]}: {sum(movies_released.values())}\n')

end_time = time.time()
runtime = end_time - start_time
print(f"Runtime: {runtime:.2f} seconds")

In [ ]:
movie_details['Genre'].head()

In [ ]:
genres=[]
for i in range(len(movie_details)):
    genres.append(movie_details['Genre'][i].split(', '))

In [ ]:
unique_genres= []
for i in range (len(genres)):
    for j in range(len(genres[i])):
        if genres[i][j] not in unique_genres:
            unique_genres.append(genres[i][j])


In [ ]:
unique_genres= list(set(unique_genres))

In [ ]:
movie_details['Genre'] = movie_details['Genre'].apply(lambda x: x.split(', '))
mlb = MultiLabelBinarizer()

genre_encoded = pd.DataFrame(mlb.fit_transform(movie_details['Genre']), columns=mlb.classes_)
movie_details_encoded = pd.concat([movie_details, genre_encoded], axis=1)

# Drop the original 'Genre' column
movie_details_encoded.drop(columns=['Genre'], inplace=True)

movie_details_encoded

In [ ]:
# Flatten the list of encoded genres into a single list
encoded_genres_flat = [genre for sublist in movie_details['Encoded_Genre'] for genre in sublist]

genre_counts = pd.Series(encoded_genres_flat).value_counts()

genre_labels = encoder.inverse_transform(genre_counts.index)

plt.figure(figsize=(10, 6))
genre_counts.plot(kind='bar', color='skyblue')
plt.title('Frequency of Different Genres')
plt.xlabel('Genre')
plt.ylabel('Frequency')
plt.xticks(range(len(genre_labels)), genre_labels, rotation=45, ha='right')  # Set genre labels as x-axis ticks
plt.tight_layout()
plt.show()
